<a href="https://colab.research.google.com/github/Infant-Joshva/AI_Agents-Learning_Materials/blob/main/Practical_Part/LangChain_RAG_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG from Youtube Transcript using LangChain

## Installing the Library

In [14]:
import os
from getpass import getpass
os.environ["GEMINI_API_KEY"] = getpass('Enter your API Key:')

Enter your API Key:··········


In [15]:
!pip install -q youtube-transcript-api langchain-community langchain-google-genai faiss-cpu

In [16]:
from IPython.terminal.embed import EmbeddedMagics
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## Indexing (Document Injection)

In [30]:
video_id = "AMZSCWxOU9M"

try:
    api = YouTubeTranscriptApi()   # create object
    transcript_list = api.fetch(video_id, languages=['en-IN'])

    transcript = " ".join([chunk.text for chunk in transcript_list])
    print (transcript)
except TranscriptsDisabled:
    print("No captions available for this video.... ⚠️")


[Intro music plays] Hello, everyone!
I’m Madan Gowri. Hello, my dear MG squad! Would you believe Tamil words were carved in Egypt’s pyramids? This is a major
recent discovery. A three-day conference
was recently held. Scientists and
archaeologists from Switzerland and
France participated. They presented evidence
of Tamil presence. Not many people are
aware of this breakthrough. Tamil names were discovered
in Egypt's Valley of the Kings. This is the sacred burial
site of Egyptian Pharaohs. Great kings were buried there
with gold and precious gems. Tamil names were
found in this very location. I will explain those names. But first, watch this. For Tamils worldwide, this
video will give you goosebumps. We often discuss
globalization today. We talk about international
trade and connectivity. We also discuss
issues that arise when leaders like
Trump come to power. But 2,000 years ago, Tamils were already practicing
globalization. Today, we have actual
proof, not just literature. It shows ho

In [31]:
transcript_list

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='[Intro music plays]', start=0.073, duration=6.561), FetchedTranscriptSnippet(text='Hello, everyone!\nI’m Madan Gowri.', start=6.658, duration=1.5), FetchedTranscriptSnippet(text='Hello, my dear MG squad!', start=8.221, duration=1.567), FetchedTranscriptSnippet(text='Would you believe Tamil words', start=9.889, duration=2.833), FetchedTranscriptSnippet(text='were carved in Egypt’s pyramids?', start=12.746, duration=2.604), FetchedTranscriptSnippet(text='This is a major\nrecent discovery.', start=15.469, duration=4.041), FetchedTranscriptSnippet(text='A three-day conference\nwas recently held.', start=19.534, duration=2.774), FetchedTranscriptSnippet(text='Scientists and\narchaeologists from', start=22.572, duration=3.511), FetchedTranscriptSnippet(text='Switzerland and\nFrance participated.', start=26.107, duration=2.708), FetchedTranscriptSnippet(text='They presented evidence\nof Tamil presence.', start=28.839, duration=2.829),

## Text Splitting

In [32]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [33]:
chunks

[Document(metadata={}, page_content="[Intro music plays] Hello, everyone!\nI’m Madan Gowri. Hello, my dear MG squad! Would you believe Tamil words were carved in Egypt’s pyramids? This is a major\nrecent discovery. A three-day conference\nwas recently held. Scientists and\narchaeologists from Switzerland and\nFrance participated. They presented evidence\nof Tamil presence. Not many people are\naware of this breakthrough. Tamil names were discovered\nin Egypt's Valley of the Kings. This is the sacred burial\nsite of Egyptian Pharaohs. Great kings were buried there\nwith gold and precious gems. Tamil names were\nfound in this very location. I will explain those names. But first, watch this. For Tamils worldwide, this\nvideo will give you goosebumps. We often discuss\nglobalization today. We talk about international\ntrade and connectivity. We also discuss\nissues that arise when leaders like\nTrump come to power. But 2,000 years ago, Tamils were already practicing\nglobalization. Today, 

In [34]:
len(chunks)

12

## Embedding generation and storing in vectore store

In [35]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

for m in genai.list_models():
    if "embed" in m.name.lower():
        print(m.name, " | ", m.supported_generation_methods)


models/gemini-embedding-001  |  ['embedContent', 'countTextTokens', 'countTokens', 'asyncBatchEmbedContent']


In [36]:
embedding = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
vectorstore = FAISS.from_documents(chunks, embedding)

In [37]:
vectorstore.index_to_docstore_id

{0: '47b1854c-ed70-4bab-99de-06370245ea2b',
 1: '4d8b31fa-a642-4efc-ba8e-fbc1db4c48eb',
 2: '3048880a-7d24-4a38-bebc-5506b04fe650',
 3: '49aeb334-17ef-4cc4-b759-c4341870ec71',
 4: 'd03c4e5a-d4f4-4a22-b755-191d7f047414',
 5: '1e75abc3-36a6-4011-826d-337a40f3b5f0',
 6: 'e4291a0a-50e6-4a8e-86ef-1bfd3f51c777',
 7: '171ae5b8-f325-4d93-8f14-584e90945b18',
 8: 'd8f883cf-7738-4fea-812e-52cc10e88701',
 9: '60307b38-d2d4-4038-91a3-a16b76edf193',
 10: '771f3d89-5df4-4112-9e01-ff273eef0efb',
 11: '95d55624-77c7-4f80-9ee6-95532d61f7d6'}

In [43]:
vectorstore.get_by_ids(['47b1854c-ed70-4bab-99de-06370245ea2b'])

[Document(id='47b1854c-ed70-4bab-99de-06370245ea2b', metadata={}, page_content="[Intro music plays] Hello, everyone!\nI’m Madan Gowri. Hello, my dear MG squad! Would you believe Tamil words were carved in Egypt’s pyramids? This is a major\nrecent discovery. A three-day conference\nwas recently held. Scientists and\narchaeologists from Switzerland and\nFrance participated. They presented evidence\nof Tamil presence. Not many people are\naware of this breakthrough. Tamil names were discovered\nin Egypt's Valley of the Kings. This is the sacred burial\nsite of Egyptian Pharaohs. Great kings were buried there\nwith gold and precious gems. Tamil names were\nfound in this very location. I will explain those names. But first, watch this. For Tamils worldwide, this\nvideo will give you goosebumps. We often discuss\nglobalization today. We talk about international\ntrade and connectivity. We also discuss\nissues that arise when leaders like\nTrump come to power. But 2,000 years ago, Tamils were

## Retrival

In [53]:
retriver = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [55]:
retriver

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x78cd43bd1580>, search_kwargs={'k': 2})

In [56]:
retriver.invoke("Where Tamil words were carved")

[Document(id='47b1854c-ed70-4bab-99de-06370245ea2b', metadata={}, page_content="[Intro music plays] Hello, everyone!\nI’m Madan Gowri. Hello, my dear MG squad! Would you believe Tamil words were carved in Egypt’s pyramids? This is a major\nrecent discovery. A three-day conference\nwas recently held. Scientists and\narchaeologists from Switzerland and\nFrance participated. They presented evidence\nof Tamil presence. Not many people are\naware of this breakthrough. Tamil names were discovered\nin Egypt's Valley of the Kings. This is the sacred burial\nsite of Egyptian Pharaohs. Great kings were buried there\nwith gold and precious gems. Tamil names were\nfound in this very location. I will explain those names. But first, watch this. For Tamils worldwide, this\nvideo will give you goosebumps. We often discuss\nglobalization today. We talk about international\ntrade and connectivity. We also discuss\nissues that arise when leaders like\nTrump come to power. But 2,000 years ago, Tamils were

## Agumentation

In [64]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)

In [66]:
prompt = PromptTemplate(

    template =
    """
    You are a helpful assistant. answer only from provided transcript context. if the content is insufficient, just say you dont known
    {context}
    Question: {question}
    """,

    input_variables=["context", "question"]
)

In [67]:
user_question = "is the language Tamil is discussed in this video? if yes then what was discussed"
retrived_docs = retriver.invoke(user_question)

In [68]:
retrived_docs

[Document(id='47b1854c-ed70-4bab-99de-06370245ea2b', metadata={}, page_content="[Intro music plays] Hello, everyone!\nI’m Madan Gowri. Hello, my dear MG squad! Would you believe Tamil words were carved in Egypt’s pyramids? This is a major\nrecent discovery. A three-day conference\nwas recently held. Scientists and\narchaeologists from Switzerland and\nFrance participated. They presented evidence\nof Tamil presence. Not many people are\naware of this breakthrough. Tamil names were discovered\nin Egypt's Valley of the Kings. This is the sacred burial\nsite of Egyptian Pharaohs. Great kings were buried there\nwith gold and precious gems. Tamil names were\nfound in this very location. I will explain those names. But first, watch this. For Tamils worldwide, this\nvideo will give you goosebumps. We often discuss\nglobalization today. We talk about international\ntrade and connectivity. We also discuss\nissues that arise when leaders like\nTrump come to power. But 2,000 years ago, Tamils were

In [69]:
context_text = "\n\n".join(doc.page_content for doc in retrived_docs)
context_text

"[Intro music plays] Hello, everyone!\nI’m Madan Gowri. Hello, my dear MG squad! Would you believe Tamil words were carved in Egypt’s pyramids? This is a major\nrecent discovery. A three-day conference\nwas recently held. Scientists and\narchaeologists from Switzerland and\nFrance participated. They presented evidence\nof Tamil presence. Not many people are\naware of this breakthrough. Tamil names were discovered\nin Egypt's Valley of the Kings. This is the sacred burial\nsite of Egyptian Pharaohs. Great kings were buried there\nwith gold and precious gems. Tamil names were\nfound in this very location. I will explain those names. But first, watch this. For Tamils worldwide, this\nvideo will give you goosebumps. We often discuss\nglobalization today. We talk about international\ntrade and connectivity. We also discuss\nissues that arise when leaders like\nTrump come to power. But 2,000 years ago, Tamils were already practicing\nglobalization. Today, we have actual\n\nidentity, without 

In [70]:
final_prompt = prompt.invoke({"context":context_text, "question":user_question})
final_prompt

StringPromptValue(text="\n    You are a helpful assistant. answer only from provided transcript context. if the content is insufficient, just say you dont known\n    [Intro music plays] Hello, everyone!\nI’m Madan Gowri. Hello, my dear MG squad! Would you believe Tamil words were carved in Egypt’s pyramids? This is a major\nrecent discovery. A three-day conference\nwas recently held. Scientists and\narchaeologists from Switzerland and\nFrance participated. They presented evidence\nof Tamil presence. Not many people are\naware of this breakthrough. Tamil names were discovered\nin Egypt's Valley of the Kings. This is the sacred burial\nsite of Egyptian Pharaohs. Great kings were buried there\nwith gold and precious gems. Tamil names were\nfound in this very location. I will explain those names. But first, watch this. For Tamils worldwide, this\nvideo will give you goosebumps. We often discuss\nglobalization today. We talk about international\ntrade and connectivity. We also discuss\nissu

## Genaration

In [71]:
answer = llm.invoke(final_prompt)

In [73]:
print(answer.content)

Yes, the language Tamil is discussed in this video.

It was discussed that:
*   Tamil words were carved in Egypt's pyramids.
*   Scientists and archaeologists from Switzerland and France presented evidence of Tamil presence.
*   Tamil names were discovered in Egypt's Valley of the Kings, the sacred burial site of Egyptian Pharaohs.
*   Tamils were practicing globalization 2,000 years ago.
*   More evidence is needed to prove Tamil was the first language.


## Do this same with Chain method

In [74]:
!pip install -q langchain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import LLMChain